# Compulsory assignment 1 - Using Dask-ml on large data
## Group #16
### Thomas Moen and Jørgen Kongsro

In [3]:
%matplotlib notebook


import numpy as np
import pandas as pd
import dask
import os


from operator import itemgetter

import numpy as np
import scipy

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import matplotlib.pyplot as plt
import skimage.io

from dask_ml.linear_model import LogisticRegression
from dask_ml.model_selection import train_test_split


ModuleNotFoundError: No module named 'dask_ml'

In [ ]:
# Install Kaggle API
# How to setup: https://github.com/Kaggle/kaggle-api
# or visit: https://adityashrm21.github.io/Setting-Up-Kaggle/

#!pip install kaggle

In [ ]:
# Download Kaggle data using Kaggle API
#!kaggle competitions download -c dat300-ca1-autumn-2019


In [ ]:
# Unzip Kaggle data
#!unzip "dat300-ca1-autumn-2019.zip" -d "/tmp/whatever"

In [4]:
# List files in directory

if os.name == 'nt':
    workdir = 'C://Users//thomoe//Documents//myDAT300//dat300-ca1-autumn-2019//'
elif os.name == 'posix':
    workdir = '/Users/jorgenkongsro/Downloads/dat300-ca1-autumn-2019/'
    
os.listdir(workdir)


['X_test.csv', 'X_train.csv', 'y_test_sampleSubmission.csv', 'y_train.csv']

In [22]:
def import_data(csv_file):
    """ Import data from csv file

    :param data: a .csv separated dataset
    :return: a pandas data array, df
    
    """
    
    
    df = dd.read_csv(csv_file)
    return df

x_train_df = import_data(workdir + 'X_train.csv')
x_test_df = import_data(workdir + 'X_test.csv')
y_train_df = import_data(workdir + 'y_train.csv')



,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162
0,-0.180510,-0.192210,-0.205150,-0.215250,-0.215960,-0.215960,-0.216130,-0.218260,-0.229070,-0.506090,...,-0.31380,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709
1,-0.164560,-0.180510,-0.205150,-0.205680,-0.206210,-0.215250,-0.216130,-0.218260,-0.229070,-0.506090,...,-0.31380,-0.16006,-0.16006,-0.16006,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709
2,-0.044046,-0.050072,-0.057870,-0.066023,-0.067086,-0.067972,-0.087645,-0.099342,-0.309710,1.230800,...,-0.13746,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924
3,0.340010,0.329020,0.312360,0.239520,0.235090,0.189360,0.173410,0.161540,0.158350,-0.038613,...,0.38480,0.42301,0.42301,0.42301,0.32402,0.32402,0.32402,0.32402,0.32402,0.32402
4,-0.155170,-0.168990,-0.179270,-0.218790,-0.251050,-0.252110,-0.288620,-0.325130,-0.346220,-0.324840,...,-0.28911,-0.14978,-0.14978,-0.14978,-0.25570,-0.25570,-0.25570,-0.25570,-0.32433,-0.32433
5,-0.156230,-0.168990,-0.179270,-0.182990,-0.218790,-0.247150,-0.252110,-0.288620,-0.325130,-0.355590,...,-0.28911,-0.14978,-0.14978,-0.15118,-0.25570,-0.25570,-0.25570,-0.25570,-0.25570,-0.32433
6,-0.126460,-0.127520,-0.136910,-0.145240,-0.150920,-0.157650,-0.159420,-0.163850,-0.173070,-0.547810,...,-0.15917,-0.15118,-0.15118,-0.15118,-0.15118,-0.15118,-0.15118,-0.15118,-0.15118,-0.15118
7,-0.127700,-0.131070,-0.132130,-0.136740,-0.137800,-0.144890,-0.144890,-0.147370,-0.160660,-0.522210,...,-0.15917,-0.15118,-0.15118,-0.15118,-0.15118,-0.15118,-0.15118,-0.15118,-0.15118,-0.16006
8,-0.154810,-0.161190,-0.165090,-0.170410,-0.170760,-0.172010,-0.173600,-0.176260,-0.184940,-0.573790,...,-0.15917,-0.15118,-0.16006,-0.16006,-0.16006,-0.16006,-0.16006,-0.16006,-0.16006,-0.16006
9,-0.047059,-0.062478,-0.064073,-0.075770,-0.084277,-0.084809,-0.085163,-0.085518,-0.091366,-0.542090,...,-0.13746,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924


In [34]:
#get dimensions det:

x_train_df_nrows = len(x_train_df)
x_train_df_ncols = len(x_train_df.columns)
x_test_df_nrows = len(x_test_df)
x_test_df_ncols = len(x_test_df.columns)


In [23]:
#inspect some rows with missing values (empty in csv file) to check that these become NaN (can be deleted)
x_train_df.head(10)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162
0,0.650080,0.307440,0.306270,0.186580,0.179110,0.178230,0.174430,0.151130,0.139410,1.837900,...,0.130790,0.119300,0.119300,0.119300,0.119300,0.119300,0.119300,0.119300,0.119300,0.119300
1,0.098689,0.090486,0.063531,0.050347,0.049322,0.039360,0.007132,-0.016892,-0.022459,-0.310620,...,-0.038232,0.040767,0.040767,0.040767,-0.003073,-0.003073,-0.003073,-0.003073,-0.003073,-0.003073
2,0.079645,0.074518,0.071149,0.052691,0.039653,0.039214,0.037016,0.029252,0.010062,-0.348400,...,-0.038232,0.040767,0.040767,0.040767,0.040767,0.040767,0.040767,0.040767,0.040767,0.040767
3,0.148200,0.101180,0.091218,0.090486,0.051666,0.038042,0.021781,0.006546,-0.008835,-0.286620,...,0.066729,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966
4,0.148200,0.101180,0.091218,0.089167,0.051666,0.043755,0.040093,0.021781,0.006546,-0.380270,...,0.066729,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966,0.051966
5,0.125640,0.117730,0.093123,0.085359,0.081110,0.046831,NaN,NaN,NaN,-0.414100,...,NaN,0.044208,0.044208,0.044208,0.044208,0.044208,0.044208,NaN,NaN,NaN
6,0.048150,0.022660,0.016801,0.013871,0.013431,-0.001364,NaN,NaN,NaN,-0.443830,...,NaN,0.044208,0.044208,0.044208,0.044208,0.044208,0.044208,NaN,NaN,NaN
7,0.149230,0.040532,0.031743,0.008744,-0.010593,-0.087940,NaN,NaN,NaN,-0.087262,...,NaN,-0.009096,-0.009096,-0.009096,-0.009096,-0.009096,-0.009096,NaN,NaN,NaN
8,0.228480,0.209440,0.149230,0.142200,0.098250,0.040532,NaN,NaN,NaN,-0.087262,...,NaN,-0.009096,-0.009096,-0.009096,-0.009096,-0.009096,-0.009096,NaN,NaN,NaN
9,0.089167,0.073493,0.069537,0.055621,0.053277,0.043755,NaN,NaN,NaN,-0.492080,...,NaN,0.103310,0.103310,0.103310,0.103310,0.051966,0.051966,NaN,NaN,NaN


In [6]:
def percent_missing(dataframe):
    """ Check for percent missing values in dataframe
    :param data: dataframe
    :return: dataframe
    
    """
    missing_values = dataframe.isnull().sum()
    
    with ProgressBar():
        percent_missing = ((missing_values / dataframe.index.size) * 100).compute()
        
    return percent_missing


#percent_missing(x_train_df)

""""
note: the results indicate that the features come in "tripets", e.g. f1 to f3 have quite similar missing%. 
We could impute some values very precisely by insert the mean of the other values within the triplet, 
if only one or two values are missing
""""

[########################################] | 100% Completed | 36.3s


f1      3.628002
f2      3.754723
f3      3.836795
f4      4.133142
f5      4.299061
f6      4.377457
f7      9.171993
f8      9.276405
f9      9.410288
f10     3.628002
f11     3.754723
f12     3.836795
f13     4.133142
f14     4.299061
f15     4.377457
f16     9.171993
f17     9.276405
f18     9.410288
f19     3.628002
f20     3.754723
f21     3.836795
f22     4.133142
f23     4.299061
f24     4.377457
f25     9.171993
f26     9.276405
f27     9.410288
f28     3.628002
f29     3.754723
f30     3.836795
          ...   
f133    9.171993
f134    9.276405
f135    9.410288
f136    3.628002
f137    3.754723
f138    3.836795
f139    4.133142
f140    4.299061
f141    4.377457
f142    9.171993
f143    9.276405
f144    9.410288
f145    3.628002
f146    3.754723
f147    3.836795
f148    4.133142
f149    4.299061
f150    4.377457
f151    9.171993
f152    9.276405
f153    9.410288
f154    3.628002
f155    3.754723
f156    3.836795
f157    4.133142
f158    4.299061
f159    4.377457
f160    9.1719

In [ ]:
nmissing = 

In [15]:
def nan_filter(dataframe):
    """ Filter out NaN
    :param data: dataframe
    :return: dataframe
    
    """
    filtered = dataframe.dropna(how='all', axis= 0, subset=None, thresh=None).compute()

    return filtered

filtered = nan_filter(x_train_df)
percent_missing(filtered)

#THIS ONE DOES NOT WORK, THE AXIS PARAMETER SEEMS TO HAVE BEEN DEPRECATED. 
#DROPPING A WITHIN SPECIFYING ROWS OR COLUMNS DID NOT REMOVE ANY OF EITHER

#remove columns where all observations are NaN: 
#print(len(x_train_df.columns))
#print(len(x_train_df))
#x_train_df_nafiltered = x_train_df.dropna(how='all').compute()
#print(len(x_train_df.columns))
#print(len(x_train_df))



TypeError: dropna() got an unexpected keyword argument 'axis'

In [46]:
def drop_columns(df, thresh_perc, ncols = None, nrows = None):
    """ Drop columns which have more than 50% missing
    :param data: dataframe
    :return: dataframe
    
    """
    
    if ncols == None: 
        ncols = len(df.columns)
        
    if nrows == None: 
        nrows = len(df)
    
    #print(df.isna())
    nmissing = list(df.isna().sum().compute())
    print(nmissing)
    colstodrop = [a for a in range(ncols) if nmissing[a]/nrows*100 >= thresh_perc]
    print(colstodrop)
    return df.drop(colstodrop, axis=1)
 
    
    #columns_to_drop = list(percent_missing[percent_missing >= 50].index)
    #data = dataframe.drop(columns_to_drop, axis=1)
    #return data
    
print(len(x_test_df.columns))
test = drop_columns(x_test_df, 80, x_test_df_ncols, x_test_df_nrows)
print(len(test.columns))
    

162
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[]
162


In [33]:
#remove columns which have more than N% NaN's: 

#thresh = 50

for k in range(5):
    print(len(x_train_df.iloc[:,k]))




3155759
3155759
3155759
3155759
3155759


In [ ]:
def drop_rows(dataframe):
    """ Drop rows if more than 0 and less than 5% missing 
    :param data: dataframe
    :return: dataframe
    
    """
    rows_to_drop = list(percent_missing[(percent_missing > 0) & (percent_missing < 5)].index)
    data_clean = dataframe.dropna(subset=rows_to_drop)

In [ ]:
def impute_rows(dataframe,feature):
    """ Impute rows if more than 5% and less than 50% missing
    :param data: dataframe, feature
    :return: dataframe
    
    """
    value = median[feature]
    df_impute[feature] = df[feature].fillna(value)
    return df_impute

In [ ]:
# Print the size of a feature
print (x_train_df.count().compute())


In [ ]:
# Build model and train

# Split data into train and validation

# JØRGEN KAN BEGYNNE BYGGE DETTE

from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from dask.diagnostics import ProgressBar

bern = BernoulliNB()
with ProgressBar():
    bern.fit(x_train_df, y_train_df)